In [8]:
import openai
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

client = openai.Client()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
vector_store = client.vector_stores.create(
    name="Tutor de Apostilas"
)    

In [10]:
file = ["files/LLM.pdf"]
file_stream = [open(f, "rb") for f in file]
file_batch = client.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [11]:
file_batch.status

'completed'

In [13]:
file_batch.file_counts

FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)

In [20]:
assistant = client.beta.assistants.create(
    name="Tutor de Apostilas",
    instructions="Você é um tutor especializado em tecnologias emergentes \
        Você sabe responder perguntas sobre LLMs como OpenAI, HuggingFace, etc \
            Caso você não encontre as respostas, seja sincero e fale que você não sabe responder",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    # tool_resources=[{"vector_store_ids": [vector_store.id]}],
    model="gpt-4-turbo-preview"

)

In [21]:
pergunta = "COnforme o documento o que é Hugging Face?"

In [22]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [23]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [24]:
# Aguarda a thread ser executada
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [25]:
# Verificar resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro: {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_kJkBCz37cqVUcQweA0X3yYRt', assistant_id='asst_tYaiSanJTW9aHEoUoTRlcRQX', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=597, file_citation=FileCitation(file_id='file-MPV9FkT5Ma8yMmZeTne8ke'), start_index=584, text='【4:0†LLM.pdf】', type='file_citation')], value='Hugging Face é destacada no documento como uma comunidade de código aberto que compilou centenas de milhares de modelos de contribuidores. Esses modelos são úteis para resolver várias tarefas específicas, como geração de texto, resumo e classificação. A documentação realça a importância de se recorrer à comunidade de código aberto para acessar modelos de linguagem, mencionando como a Hugging Face e outras comunidades semelhantes estão se aproximando dos desempenhos de modelos proprietários, embora ainda não tenham alcançado o desempenho de modelos de nível extremo, como o GPT-4【4:0†LLM.pdf】.'), type='text')], creat

In [26]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [27]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_cals":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        # print(message.content[0].text.value)


====Step tool_calls

====Step message_creation
Hugging Face é destacada no documento como uma comunidade de código aberto que compilou centenas de milhares de modelos de contribuidores. Esses modelos são úteis para resolver várias tarefas específicas, como geração de texto, resumo e classificação. A documentação realça a importância de se recorrer à comunidade de código aberto para acessar modelos de linguagem, mencionando como a Hugging Face e outras comunidades semelhantes estão se aproximando dos desempenhos de modelos proprietários, embora ainda não tenham alcançado o desempenho de modelos de nível extremo, como o GPT-4【4:0†LLM.pdf】.


In [28]:
print(mensagens.data[0].content[0].text.value)

Hugging Face é destacada no documento como uma comunidade de código aberto que compilou centenas de milhares de modelos de contribuidores. Esses modelos são úteis para resolver várias tarefas específicas, como geração de texto, resumo e classificação. A documentação realça a importância de se recorrer à comunidade de código aberto para acessar modelos de linguagem, mencionando como a Hugging Face e outras comunidades semelhantes estão se aproximando dos desempenhos de modelos proprietários, embora ainda não tenham alcançado o desempenho de modelos de nível extremo, como o GPT-4【4:0†LLM.pdf】.
